# Growing degree days & Extreme degree days calculations

## Packages

In [219]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import dask
import os

## Functions

In [363]:
def degreeDays(single_day, degree_day_method, kelvin_starting=True):
    '''
    Calculates growing degree days, using a piecewise function, ASSUMING KELVIN STARTING
    
    inputs:
    dataset with tmax & tmin as variables as single_day
    degree_day_method, which is either gdd or edd, 10.0 degrees or 29.0 degrees respectively (in degree celsius)
    outputs:
    growing degree days for dataset
    (Haqiqi et al. 2021; D’Agostino and Schlenker, 2015)
    '''
    
    single_day = single_day - 273.15

    
    if degree_day_method == 'gdd':
        corn_baseline = 10.0
    elif degree_day_method == 'edd':
        corn_baseline = 29.0
    else:
        raise Exception(f'Invalid degree_day_method "{degree_day_method}", please specify either "gdd" or "edd"')
    gdd_ds = xr.full_like(single_day.tmax, 0.0)

    # b <= tmin
    b_lower_tmin = (single_day.tmin + single_day.tmax) / 2 - corn_baseline
    gdd_ds = xr.where(single_day.tmin >= corn_baseline, b_lower_tmin, gdd_ds)
    
    # tmin < b <= tmax
    arccos_arg = (2 * corn_baseline - single_day.tmax - single_day.tmin) / (single_day.tmax - single_day.tmin)
    arccos_arg = arccos_arg.clip(min=-1, max=1)
    
    t_bar = np.arccos(arccos_arg)
    tmin_lower_b_lower_tmax = (
        t_bar / np.pi * ((single_day.tmax + single_day.tmin) / 2 - corn_baseline)
        + (single_day.tmax - single_day.tmin) / (2 * np.pi) * np.sin(t_bar)
    )
    
    gdd_ds = xr.where((single_day.tmin < corn_baseline) & (corn_baseline <= single_day.tmax), tmin_lower_b_lower_tmax, gdd_ds)
    
    # tmax < b 
    gdd_ds = xr.where((single_day.tmax < corn_baseline), 0, gdd_ds)
    
    return gdd_ds

def yearlyCalculationSum(year, filein_base_yearly):
    '''
    calculates growing degree days, & extreme degree days from a year & file_path.
    This is for the growing season (April 1 to September 31), per Haqiqi et al. 2020
    
    inputs:
    year
    filein_base_yearly that with daily tmax & tmin values
    outputs:
    yearly summed gdd & edd as nc file, saved @ some location as single year nc file
    
    '''
    start_dt = datetime.datetime(year, 4, 1, 0, 0)
    end_dt = datetime.datetime(year, 9, 30, 0, 0)
    
    first_file = f"{filein_base_yearly}/tmp_downloads/{year}/NLDAS_FORA0125_H.A{year}0701.nc"
    first_file_ds = xr.open_dataset(first_file)
    gdd_first = degreeDays(first_file_ds, 'gdd')
    zeros_first_file = xr.full_like(gdd_first, 0.0)
    
    gdd_running = zeros_first_file.copy(deep=True).drop_vars('time', errors='ignore')
    edd_running = zeros_first_file.copy(deep=True).drop_vars('time', errors='ignore')
    
    while start_dt <= end_dt:
        yyyymmdd = start_dt.strftime("%Y%m%d")
        filein_i = f"{filein_base_yearly}/tmp_downloads/{year}/NLDAS_FORA0125_H.A{yyyymmdd}.nc"
        dataset_i  = xr.open_dataset(filein_i)
        gdd = degreeDays(dataset_i, 'gdd').drop_vars('time', errors='ignore')
        edd = degreeDays(dataset_i, 'edd').drop_vars('time', errors='ignore')
        
        gdd_running = gdd_running + gdd
        edd_running = edd_running + edd

            
        start_dt += datetime.timedelta(days=1)

    combined_dataset = xr.Dataset({
        "gdd": gdd_running,
        "edd": edd_running
    })

    try:
        os.mkdir(f"{filein_base_yearly}/gdd_edd/")
    except Exception:
        pass
        
    combined_dataset.to_netcdf(f"{filein_base_yearly}/gdd_edd/gdd_edd_NLDAS_FORA0125_H.A{year}.nc")
    
    return

In [362]:
filein_base_yearly = "/storage/home/cta5244/work/pyWBM_yield_data/NCEPNARR_NLDAS_Hist_Temp"
start_year = 1979
end_year = 2025

In [126]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="2GiB",
    walltime="03:00:00",
)

cluster.scale(jobs=50) 

In [132]:
from dask.distributed import Client

client = Client(cluster)

In [135]:
client

<Client: 'tcp://146.186.150.11:43259' processes=50 threads=50, memory=100.00 GiB>

In [359]:
results = []
for year in np.arange(start_year, end_year, 1):
    out = dask.delayed(yearlyCalculationSum)(year=year, filein_base_yearly=filein_base_yearly)
    results.append(out)

In [360]:
results = dask.compute(*results)